In [1]:
from lavis.datasets.builders import load_dataset
from lavis.models import load_model_and_preprocess
from dotenv import load_dotenv
FLICKR_PATH='/media/caduser/MyBook/chau/.cache/flickr30k/images'
model, vis_processors, txt_processors = load_model_and_preprocess("blip_retrieval", "coco", is_eval=False)
dataset = load_dataset("flickr30k", vis_path=FLICKR_PATH, cfg_path=None)


Using downloaded and verified file: /media/caduser/MyBook/chau/.cache/flickr30k/annotations/train.json
Using downloaded and verified file: /media/caduser/MyBook/chau/.cache/flickr30k/annotations/val.json
Using downloaded and verified file: /media/caduser/MyBook/chau/.cache/flickr30k/annotations/test.json


In [3]:
dataset['train'][0]['']

{'image': <PIL.Image.Image image mode=RGB size=333x500>,
 'text_input': 'Two young guys with shaggy hair look at their hands while hanging out in the yard.',
 'image_id': 0,
 'instance_id': '0'}